# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
# Given imports
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
from scipy.sparse import find

# Import pickle to open processed courses
import pickle as pk
# Use Sparks sparse vectors
from pyspark.mllib.linalg import Vectors 

from utils import load_pkl, listPrettyPrint
from lab04_helper import *

## Exercise 4.8: Topics extraction

In [ ]:
# Let's open the pickle to load it.
processedCourses = load_pkl("cidWithBag")

# We also need the list of words
uniqueWords = load_pkl("indexToWord")

# Get the course indexes for the matrix
courseIndices = load_pkl("indexToCourse")

# Get the TF-IDF matrix
TF_course_matrix = load_sparse_csr("TFIDF.npz")

In [ ]:
# Transform the list of words into a spark vector usig TF-IDF
#### Use snippets Get value with index from RDD and Spar's sparse vectors

# Create an RDD we will fill with our sparse vectors 
v_dim = len(uniqueWords)
vector_list = []

# Create the sparse vectors using the matrix.
for i in range (len(courseIndices)):
    values = find(TF_course_matrix.getcol(i))[-1]
    indices = (TF_course_matrix.getcol(i).nonzero())[0] # Could use first of find though
    v = Vectors.sparse(v_dim, indices, values)
    vector_list.append((i, v))
    
# In the RDD we need a tuple wit (index of document, corresponding SparseVector) then map(list)
#docs = courseIndices.zipWithIndex(vector_list)
#print(docs)

# Create the RDD (corpus) from the list
corpus = sc.parallelize(vector_list)
#print(corpus.take(2))
corpus = corpus.map(list)

# Cluster the documents into ten topics using LDA
ldaModel = LDA.train(corpus, k=10)

In [ ]:
def printTopics(model, maxTerms):
    # Gives a list of the 10 most linked words to our topics
    topic_indices = model.describeTopics(maxTermsPerTopic=maxTerms)

    # Extraire les indices (premier tuple) de la liste de mots
    topic = 1
    for i in topic_indices:
        print("Topic #", topic)
        listPrettyPrint([str("%s - %.7f" % (uniqueWords[i[0][j]], i[1][j]))  for j in range(10)], 3)
        print('\n')
        topic += 1

In [ ]:
printTopics(ldaModel, 10)

* **Topic 1** : Materials physics
* **Topic 2** : IC students mental health
* **Topic 3** : Physics
* **Topic 4** : (Risk) Management
* **Topic 5** : Chemistry
* **Topic 6** : Urban architecture
* **Topic 7** : Financial Engineering
* **Topic 8** : Circuits/Electricity (including low level as CMO)
* **Topic 9** : Biology
* **Topic 10** : Computing

TODO : Compare to LSI

## Exercise 4.9: Dirichlet hyperparameters

#### Varying the prior on topic distribution α

In [ ]:
topic_indices_list = []
for i in range(2, 11):
    model = (LDA.train(corpus, k=10, seed=2, docConcentration=float(i), topicConcentration=1.01))
    topic_indices_list.append(model.describeTopics(maxTermsPerTopic=10))

In [ ]:
for topic in range(1,10):
    print("\n\t\tTOPIC %d beta=1.01\n" %(topic))
    alpha = 2
    print
    for i in range(0, 9):
        print("\n\t alpha=%d \n" %(alpha))
        j = topic_indices_list[i][topic-1]
        listPrettyPrint([str("%s - %.7f" % (uniqueWords[j[0][k]], j[1][k]))  for k in range(10)], 3)
        alpha += 1
        print('\n')

TODO: Explain wtf is happening here.

#### Varying the prior on word distribution per topic β

In [ ]:
topic_indices_list = []
array = [1.01, 1.02, 1.03, 104, 1.05, 1.06, 1.07, 1.08, 1.09, 2.0]

for i in array:
    model = (LDA.train(corpus, k=10, seed=2, docConcentration=6.0, topicConcentration=float(i)))
    topic_indices_list.append(model.describeTopics(maxTermsPerTopic=10))

In [ ]:
for topic in range(1,10):
    print("\n\t\tTOPIC %d alpha=6\n" %(topic))
    beta_index = 0
    print
    for i in range(0, 9):
        print("\n\t beta=%d \n" %(array[beta_index]))
        j = topic_indices_list[i][topic-1]
        listPrettyPrint([str("%s - %.7f" % (uniqueWords[j[0][k]], j[1][k]))  for k in range(10)], 3)
        alpha += 1
        print('\n')

TODO: Explain WTF is happeing here

## Exercise 4.10: EPFL's taught subjects

## Exercise 4.11: Wikipedia structure